### Instructions for building field maps

1. Begin with a blank template like:

```python
# Oregon
state = fieldmaps.State('OR')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()
```

  * The API and Date fields can be strings or lists of strings (which will be evaluated in the order given).
  * If the date is in JavaScript epoch time, add `webdate = True` to the state object declaration
2. Fill in the fields (except status_map and type_map) from the provided list
3. Change the second to last line to: `wells = fieldmaps.Dataset(state, layout)`
4. Overwrite the status_map and type_map lines with the output, and fill in the appropriate values

A (likely incomplete) list of valid types:
well_types = {'OIL',
    'GAS',
    'OILANDGAS',
    'INJECTION',
    'STORAGE',
    'DISPOSAL',
    'SERVICE',
    'DRY HOLE',
    'OBSERVATION',
    'TEST',
    'WATER',
    'OTHER',
    'UNKNOWN'}
well_statuses = {'ACTIVE', 'INACTIVE', 'A', 'PA', 'TA', 'SI', 'DRY', 'ORPHAN', 'PERMITTED', 'CANCELLED', 'UNKNOWN'} # PA: plugged and abandoned, TA: temporarily abandoned
well_categories = {'CONVENTIONAL','UNCONVENTIONAL','FRAC','CBM','SERVICE','TEST','STORAGE','WATER','OTHER','UNKNOWN'}


In [2]:
# module that imports CSVs and processes to DB
import os, errno, array, csv, math, re
from datetime import datetime
import dateutil.parser
import importlib
import psycopg2
import fieldmaps

In [3]:
# Alabama
import fieldmaps.al
state = fieldmaps.State('AL', webdate = True)
well_source_url = 'http://www.gsa.state.al.us/ogb/results.aspx?api=%s&permit=%s'
state.source_url = 'http://www.gsa.state.al.us/ogb/db_main.html'
state.description = """
Data from the Geological Survey of Alabama State Oil and Gas Board
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.al.layout)
db_result = dataset.write_to_db()

Wrote description for AL to database
16414 of 17086 rows loaded
first item: [{'state': 'AL', 'api': '01297203230000', 'lon': -88.302816, 'lat': 30.245575, 'date': datetime.date(1991, 7, 29), 'type': 'GAS', 'status': 'PA', 'category': 'CONVENTIONAL', 'description': 'Permit Num: 10008-OS-52-BH\nWell Name: STATE LEASE 527 #1\nOperator: Arco Oil & Gas Co., Div. Of Atlantic Richfield'}]
loaded 16414 of 16414 records into the database


In [5]:
# Alaska
import fieldmaps.ak
state = fieldmaps.State('AK')
state.projection = 4267 # NAD27
state.source_url = 'http://aogweb.state.ak.us/DataMiner3/Forms/WellList.aspx'
state.description = """
Data from the Alaska Oil and Gas Conservation Commission
"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ak.layout)
dataset.write_to_db()

Wrote description for AK to database
8911 of 9160 rows loaded
first item: [{'state': 'AK', 'api': '50103205776300', 'lon': -151.043238764074, 'lat': 70.3376726847063, 'date': datetime.date(2018, 3, 13), 'type': 'OIL', 'status': 'UNKNOWN', 'category': 'CONVENTIONAL', 'description': 'Permit No. 218026\nOperator: CONOCOPHILLIPS ALASKA, INC.\nArea: ARCTIC SLOPE\nField: COLVILLE RIVER\nPools: QANNIK  OIL\nWell Name: COLVILLE RIV QANN CD2-463L2-01'}]
loaded 8911 of 8911 records into the database


8911

In [7]:
# Arkansas
import fieldmaps.ar
state = fieldmaps.State('AR')
state.source_url = 'http://www.noggateway.org/reports'
state.description = "Data from the National Oil and Gas Gateway"
state.write_to_db()

state.projection = 4269 # NAD83

dataset = fieldmaps.Dataset(state, fieldmaps.ar.layout)
dataset.write_to_db()

Wrote description for AR to database
30318 of 52490 rows loaded
first item: [{'state': 'AR', 'api': '03131110170000', 'lon': -94.2957, 'lat': 35.2014, 'date': datetime.date(2003, 9, 8), 'type': 'GAS', 'status': 'ACTIVE', 'category': 'CONVENTIONAL', 'description': 'WELL NAME: Harkreader  2-15\nWELL NUMBER: 2-15\nOPERATOR: Xto Energy, Inc.\nCOUNTY: Sebastian\nFIELD NAME: B-44 Area'}]
loaded 30318 of 30318 records into the database


30318

In [10]:
# Arizona
import fieldmaps.az
state = fieldmaps.State('AZ')
state.source_url = 'http://services.azgs.az.gov/arcgis/rest/services/aasggeothermal/AZWellHeaders/MapServer/0'
state.description = """
Data from the Arizona Oil and Gas Conservation Commission ArcGIS map at http://ogviewer.azdeq.gov/.

Dates do not seem to be available from this source.
"""
state.date_function = lambda x: None
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.az.layout)
db_result = dataset.write_to_db()

Wrote description for AZ to database
1144 of 4774 rows loaded
first item: [{'state': 'AZ', 'api': '0201290001', 'lon': -113.6326688, 'lat': 33.49979433, 'date': None, 'type': 'TEST', 'status': 'PA', 'category': None, 'description': 'Well Name: Phillips Petroleum H01 Federal\nOperator: Phillips Petroleum\nCounty: LA PAZ\nField: \nAZ OGCC Notes: no logs'}]
loaded 1144 of 1144 records into the database


In [11]:
# California
import fieldmaps.ca
state = fieldmaps.State('CA', filename = 'california-data.csv', prepend_api = True, webdate = True)
state.source_url = 'http://spatialservices.conservation.ca.gov/arcgis/rest/services/DOMS/Wells/MapServer/0'
state.description = """California Division of Oil, Gas, & Geothermal Resources 
at http://www.conservation.ca.gov/dog. Type codes: ftp://ftp.consrv.ca.gov/pub/oil/GIS/ReadMe.pdf"""
state.write_to_db()

dataset = fieldmaps.Dataset(state, fieldmaps.ca.layout)
dataset.write_to_db()

Wrote description for CA to database
226634 of 226634 rows loaded
first item: [{'state': 'CA', 'api': '0402900001', 'lon': -118.95876, 'lat': 35.43406, 'date': datetime.date(2008, 11, 29), 'type': 'OILANDGAS', 'status': 'ACTIVE', 'category': None, 'description': 'LeaseName: \nOpName: Chevron U.S.A. Inc.\nURL: https://secure.conservation.ca.gov/WellSearch/Details?api=02900001\nComments: \nFieldName: Kern River'}]
loaded 226634 of 226634 records into the database


226634

In [12]:
# Colorado
import fieldmaps.co
state = fieldmaps.State('CO')

wells = fieldmaps.Dataset(state, fieldmaps.co.layout)
wells.write_to_db()

114537 of 114537 rows loaded
first item: [{'state': 'CO', 'api': '0500105000', 'lon': -103.727484, 'lat': 39.741587, 'date': datetime.date(1957, 10, 15), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': 'Operator TOMBERLIN* BILL\nWell_Title 1 UPRR-JOLLY'}]
loaded 114537 of 114537 records into the database


114537

In [ ]:
# Connecticut - No Oil or Gas Drilling

In [ ]:
# Delaware - No Oil or Gas Drilling

In [ ]:
# District of Columbia - No Oil or Gas Drilling

In [13]:
# Florida
state = fieldmaps.State('FL', webdate = True,  prepend_api = True)
import fieldmaps.fl

wells = fieldmaps.Dataset(state, fieldmaps.fl.layout, quiet = True)
wells.write_to_db()

1267 of 1444 rows loaded
first item: [{'state': 'FL', 'api': '09912310003', 'lon': -83.45759326189415, 'lat': 30.132361018336038, 'date': datetime.date(1949, 12, 8), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 1267 of 1267 records into the database


1267

In [ ]:
# Georgia - No oil or gas activity

In [ ]:
# Hawaii - No oil or gas activity

In [14]:
# Idaho
state = fieldmaps.State('ID')
# import fieldmaps.id
state.load_data()
layout = {
    'api_field': 'API',
    'date_field': 'Date',
    'lat_field': 'lattitudeWGS84',
    'lon_field': 'longitudeWGS84',
    'status_field': 'Status',
    'status_map': {
        'Not Drilled - Permit Expired': 'CANCELLED',
        'Permitted Well': 'PERMITTED',
        'Expired Permit': 'CANCELLED',
        'Plugged and Abandoned': 'PA',
        'Shut In': 'SI',
        'Not Drilled': 'PERMITTED',
        'Drilling': 'ACTIVE',
        'Producing': 'ACTIVE',
        'Completed': 'ACTIVE',
    },
    'type_field': 'Well Type',
    'type_map': {
        '': 'UNKNOWN',
        'Expired Location': 'UNKNOWN',
        'Oil Well': 'OIL',
        'Oil & Gas': 'OILANDGAS',
        'Gas Well': 'GAS',
        'Dry Hole': 'DRY HOLE',
    }
}
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

171 of 200 rows loaded
first item: [{'state': 'ID', 'api': '1108100001', 'lon': -111.22, 'lat': 43.8, 'date': datetime.date(1903, 1, 16), 'type': 'UNKNOWN', 'status': 'PA', 'category': None, 'description': None}]
loaded 171 of 171 records into the database


171

In [15]:
# Illinois
state = fieldmaps.State('IL', webdate = True, filename = 'il-data-old.csv')
import fieldmaps.il
wells = fieldmaps.Dataset(state, fieldmaps.il.layout, quiet = True)
wells.write_to_db()

195735 of 195735 rows loaded
first item: [{'state': 'IL', 'api': '120010000200', 'lon': -90.92110000000001, 'lat': 40.080721, 'date': datetime.date(1945, 12, 31), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 195735 of 195735 records into the database


195735

In [16]:
# Indiana
state = fieldmaps.State('IN', prepend_api = True)
import fieldmaps.indiana
wells = fieldmaps.Dataset(state, fieldmaps.indiana.layout)
wells.write_to_db()

49741 of 50714 rows loaded
first item: [{'state': 'IN', 'api': '1316735', 'lon': -87.5878048752559, 'lat': 37.87489933916674, 'date': None, 'type': 'OIL', 'status': 'ACTIVE', 'category': None, 'description': 'County: Vanderburgh    \nField Name: Barker                   \nIGS ID: 16735\nIGS Reported Status: Active\nLink: <a href="https://igs.indiana.edu/pdms/wellEvents.cfm?igsID=16735">IGS link</a>'}]
loaded 49741 of 49741 records into the database


49741

In [ ]:
# Iowa - No oil or Gas

In [23]:
# Kansas
state = fieldmaps.State('KS', webdate = True)
state.projection = 4269 # NAD83

import fieldmaps.ks
layout = fieldmaps.ks.layout
wells = fieldmaps.Dataset(state, layout, quiet = True)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'API_NUMBER', 'API_WELL_NUMBER', 'API_WORKOVERS', 'COMPLETION_DATE', 'COMPLETION_DATE_TXT', 'COMPLETION_YEAR', 'COUNTY', 'COUNTY_CODE', 'CURR_OPERATOR', 'ELEVATION_DF', 'ELEVATION_GL', 'ELEVATION_KB', 'FEET_EAST_FROM_REFERENCE', 'FEET_NORTH_FROM_REFERENCE', 'FIELD_KID', 'FIELD_NAME', 'KID', 'LEASE_NAME', 'NAD27_LATITUDE', 'NAD27_LONGITUDE', 'NAD83_LATITUDE', 'NAD83_LONGITUDE', 'OBJECTID', 'OPERATOR_KID', 'OPERATOR_NAME', 'ORIG_FID', 'PERMIT_DATE', 'PERMIT_DATE_TXT', 'PLUG_DATE', 'PLUG_DATE_TXT', 'PRINCIPAL_MERIDIAN', 'PRODUCING_FORMATION', 'RANGE', 'RANGE_DIRECTION', 'REFERENCE_CORNER', 'ROTARY_TOTAL_DEPTH', 'SECTION', 'SPOT', 'SPUD_DATE', 'SPUD_DATE_TXT', 'STATE_CODE', 'STATUS', 'STATUS_TXT', 'SUBDIVISION_1_LARGEST', 'SUBDIVISION_2', 'SUBDIVISION_3', 'SUBDIVISION_4_SMALLEST', 'TOWNSHIP', 'TOWNSHIP_DIRECTION', 'WELL_CLASS', 'WELL_LABEL', 'WELL_NAME', 'WELL_TYPE'

Example rows: [OrderedDict([('', '0'), ('API_NUMBER', '15-007-10179'), (

461547

In [40]:
# Kentucky
state = fieldmaps.State('KY')
state.projection = 4267 # NAD27

import fieldmaps.ky
wells = fieldmaps.Dataset(state, fieldmaps.ky.layout)
wells.write_to_db()

117639 of 145745 rows loaded
first item: [{'state': 'KY', 'api': '16195060750000', 'lon': -82.5155, 'lat': 37.3224, 'date': datetime.date(2009, 6, 27), 'type': 'GAS', 'status': 'ACTIVE', 'category': None, 'description': 'WELL NAME: Equitable Production Company 567990\nWELL NUMBER: 567990\nOPERATOR: Eqt Production Company\nCOUNTY: Pike\nFIELD NAME: '}]
loaded 117639 of 117639 records into the database


117639

In [59]:
# Louisiana

state = fieldmaps.State('LA', webdate = True)
import fieldmaps.la

wells = fieldmaps.Dataset(state, fieldmaps.la.layout, quiet = True)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'API_NUM', 'AREA_USDW_VALUE', 'CLASSIFICATION', 'COASTAL_PERMIT_NUM', 'COMMENTS', 'DIRWELL', 'DISTRICT_CODE', 'DOC_ACCESS', 'EFFECTIVE_DATE', 'EXEMPT_29E_FLAG', 'FIELD_ID', 'FIELD_NAME', 'FORMATIONS', 'GIS_UPD_DATE', 'HYPERLINK', 'INJECTION_RATE', 'INJECTION_VOLUME', 'INSPECTION_DATE', 'INSPECTION_YEAR_INDEX', 'LAST_RECOMPLETION_DATE', 'LAST_TEST_DATE', 'LEASE_NUM', 'LEGEND', 'LEGEND_DESC', 'LOCATION', 'LOG_REVIEWED_FLAG', 'LOWER_PERFS', 'LUW_CODE', 'LUW_NAME', 'LUW_TYPE_CODE', 'LW_REC_STATUS', 'MAX_SIP', 'MEASURED_DEPTH', 'MERIDIAN', 'MINERAL_INTEREST_FLAG', 'MUD_DENSITY', 'OBJECTID', 'OPERATION_TYPE_CODE', 'ORGANIZATION_ID', 'ORGOP_LINE_ID', 'ORG_OPER_NAME', 'ORIGINAL_COMPLETION_DATE', 'ORPHANED_FLAG', 'ORPHAN_STATUS_CODE', 'ORPHAN_STATUS_DATE', 'PARISH_CODE', 'PARISH_NAME', 'PERMIT_DATE', 'PRODUCT_TYPE_CODE', 'RESERVOIRS', 'SANDS', 'SCOUT_DETAIL', 'SCOUT_MEASURED_DEPTH', 'SCOUT_REPORT_DATE', 'SCOUT_TRUE_VERTICAL_DEPTH', 'SCOUT_WELL

214470

In [42]:
# Maine - No Oil or Gas

In [63]:
# Maryland - gas only
state = fieldmaps.State('MD', prepend_api = True)
layout = {
    'api_field': 'Id',
    'date_field': 'Date_drill',
    'lat_field': 'Y',
    'lon_field': 'X',
    'status_field': 'Status',
    'status_map': {
        '?': 'UNKNOWN',
        'Abandoned': 'A',
        'Active': 'ACTIVE',
        'Plugged': 'PA',
        'Plugged?': 'PA',
        'plugged': 'PA',
    },
    'type_field': 'Status',
    'type_map': {
        '?': 'GAS',
        'Abandoned': 'GAS',
        'Active': 'GAS',
        'Plugged': 'GAS',
        'Plugged?': 'GAS',
        'plugged': 'GAS',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'X', 'Y', 'Id', 'Date_drill', 'Date_Plugg', 'Depth', 'Formation', 'Permit_Num', 'Permittee', 'Completion', 'Well_Depth', 'Show_of_Ga', 'Common_Nam', 'Status', 'Dry_hole', 'POINT_X', 'POINT_Y', 'Comp_Rpt'

Example rows: [OrderedDict([('X', '-79.37919687'), ('Y', '39.39788269'), ('Id', '0'), ('Date_drill', '7/1/51'), ('Date_Plugg', 'September of 1952'), ('Depth', '3602'), ('Formation', 'Oriskeny'), ('Permit_Num', '7977'), ('Permittee', 'Mid-Atlantic Oil and Gas Co.'), ('Completion', 'yes'), ('Well_Depth', '3602'), ('Show_of_Ga', '3523'), ('Common_Nam', 'Grimes No. 2'), ('Status', 'Plugged'), ('Dry_hole', 'Unknown'), ('POINT_X', '195082.9925'), ('POINT_Y', '194847.897'), ('Comp_Rpt', '<a href="L:/MINING/GIS_Shapefiles/Historic_Reports/hopkins-series.pdf#page=44">Completion Report</a>')]), OrderedDict([('X', '-79.32807733'), ('Y', '39.43129633'), ('Id', '0'), ('Date_drill', '11/9/52'), ('Date_Plugg', 'na'), ('Depth', '5195'), ('Formation', 'O

8

In [44]:
# Massachusetts - No oil or gas

In [68]:
# Michigan
# the date fields appear to be all empty. may require some supplemental scraping
state = fieldmaps.State('MI')

import fieldmaps.mi
wells = fieldmaps.Dataset(state, fieldmaps.mi.layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'', 'BH_CNTY', 'CNTY_NAME', 'OBJECTID', 'PRUName', 'PRUNumber', 'PermDate', 'PlugDate', 'Slant', 'TRS', 'api_wellno', 'bh_lat', 'bh_long', 'bh_mgr_x', 'bh_mgr_y', 'bore_type', 'cnty_fips', 'co_name', 'completion', 'concen_h2s', 'deep_fm', 'deep_fm_cd', 'dtd', 'elev_ref', 'fed_land', 'fieldname', 'fieldtype', 'kotvd', 'lease_name', 'op_no', 'permit_no', 'prod_fm', 'prod_fm_cd', 'qqqs', 'qqs', 'qs', 'ref_tops', 'sidetrck', 'state_land', 'surf_owner', 'tvd', 'twp_name', 'well_no', 'well_stat', 'well_type'

Example rows: [OrderedDict([('', '0'), ('BH_CNTY', ''), ('CNTY_NAME', ''), ('OBJECTID', '1'), ('PRUName', ''), ('PRUNumber', ''), ('PermDate', ''), ('PlugDate', ''), ('Slant', 'V'), ('TRS', ''), ('api_wellno', '21001003199000'), ('bh_lat', '44.7351'), ('bh_long', '-83.45521'), ('bh_mgr_x', '701317.24'), ('bh_mgr_y', '467974.61'), ('bore_type', 'Pre-Permit'), ('cnty_fips', ''), ('co_name', 'Kurrasch  H A'), ('completion', '0'), ('concen_h2s

76944

In [ ]:
# Minnesota - no oil or gas

In [69]:
# Mississippi
state = fieldmaps.State('MS')
layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Unassigned': 'UNKNOWN',
        'Abandoned': 'A',
        'Abandoned:Plugged': 'PA',
        'Abandoned:Dry Hole': 'A',
        'Active:Producing': 'ACTIVE',
        'Permit:Active': 'ACTIVE',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Active:Completed': 'ACTIVE',
        'Active': 'ACTIVE',
        'Inactive': 'A',
        'Active:Injecting': 'ACTIVE',
    },
    'type_map': {
        'Other': 'OTHER',
        'Gas': 'GAS',
        'Unassigned': 'OTHER',
        'Oil': 'OIL',
        'Observation Monitor Strat Test': 'TEST',
        'Gas Storage': 'STORAGE',
        'Water Supply': 'WATER',
        'Permit': 'UNKNOWN',
        'Underground Injection Control:Class 2 Disposal': 'INJECTION',
        'Dry Hole': 'DRY HOLE',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Underground Injection Control:Class 1': 'INJECTION',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Invalid layout. Available source file columns: 
'API NUMBER', 'WELL NAME', 'WELL NUMBER', 'OPERATOR', 'STATE', 'COUNTY', 'COMPLETED FORMATION NAME', 'ENCOUNTERED FORMATION NAME', 'STANDARD WELL TYPE DESCRIPTION', 'STANDARD WELL STATUS DESCRIPTION', 'WELLHEAD LATITUDE, DECIMAL DEGREES', 'WELLHEAD LONGITUDE, DECIMAL DEGREES', 'MEASURED DEPTH, FEET', 'TOTAL VERTICAL DEPTH, FEET', 'DRILLING ORIENTATION', 'BASIN NAME', 'FIELD NAME', 'STATE SUBDIVISION', 'FIRST PRODUCTION DATE', 'SPUD DATE', 'FIRST COMPLETION DATE', 'TOP WELL PERFORATION, FEET', 'BOTTOM WELL PERFORATION, FEET', 'FORMATION TOP, FEET', 'FORMATION BOTTOM, FEET', 'IN FRACFOCUS', 'UNDERGROUND SOURCE OF DRINKING WATER', 'LITHOLOGY', 'STATE WELL TYPE', 'STATE WELL STATUS', 'BOTTOM HOLE LATITUDE, DECIMAL DEGREES', 'BOTTOM HOLE LONGITUDE, DECIMAL DEGREES', 'GEODETIC DATUM', 'WELL LOCATION METHOD', 'ELEVATION', 'ELEVATION REFERENCE POINT', 'SECTION', 'TOWNSHIP', 'TOWNSHIP DIRECTION', 'RANGE', 'RANGE DIRECTION', 'MERIDIAN', 'FOOTAGE NO

22033

In [83]:
# Missouri
state = fieldmaps.State('MO', strip_whitespace = ['Well Status', 'Well Type'], prepend_api = True)
#import fieldmaps.ks
layout = {
    'api_field': 'API Number',
    'date_field': ['Spud Date','Well Status Date'],
    'lat_field': 'Well Latitude Decimal',
    'lon_field': 'Well Longitude Decimal',
    'status_field': 'Well Status',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned, Known Location and Verified': 'A',
        'Abandoned, No evidence of existence/ Unable to find': 'A',
        'Abandoned, Unknown Location': 'A',
        'Active Well': 'ACTIVE',
        'Inactive Ð Initial 90 day window': 'TA',
        'Plugged - Approved': 'PA',
        'Proposed Well (Associated with permit application only)': 'UNKNOWN',
        'Shut in - Complete': 'SI',
        'Shut in - Extended': 'SI',
        'Shut in - Incomplete': 'SI',
        'Temporarily Abandoned(Idle)': 'TA',
        'Under Construction': 'PERMITTED',
        'Unknown Well Status': 'UNKNOWN',
        'Water Well Conversion': 'UNKNOWN',
    },
    'type_field': 'Well Type',
    'type_map': {
        'Dry Hole': 'DRY HOLE',
        'Gas Storage Wells': 'STORAGE',
        'Gas(Coalbed Methane)': 'GAS',
        'Gas(Convertional, Commercial)': 'GAS',
        'Gas(Private Use)': 'GAS',
        'Horizontal Oil Well': 'OIL',
        'Injection(Class II Disposal)': 'INJECTION',
        'Injection(Class II EOR)': 'INJECTION',
        'Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Stratigraphic Test': 'TEST',
        'Stratigraphic Test, None Oil/Gas Related(has geophysical log)': 'TEST',
        'Unknown Well Type': 'UNKNOWN',
        'Water Well': 'WATER',
        'Water Well, None Oil/Gas Related(has geophysical log)': 'WATER',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

10287 of 10471 rows loaded
first item: [{'state': 'MO', 'api': '2400100001', 'lon': -92.37907, 'lat': 40.25678, 'date': datetime.date(1939, 9, 1), 'type': 'DRY HOLE', 'status': 'A', 'category': None, 'description': None}]
loaded 10287 of 10287 records into the database


10287

In [22]:
# Montana
state = fieldmaps.State('MT')
#import fieldmaps.ks
layout = {
   'date_field': 'Dt_Cmp',
    'api_field': 'API #',
    'type_field': 'Well_Type',
    'status_field': 'Wl_Status',
    'lat_field': 'Wh_Lat',
    'lon_field': 'Wh_Long',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned - Unapproved': 'A',
        'Active Injection': 'ACTIVE',
        'Completed': 'ACTIVE',
        'Domestic': 'ACTIVE',
        'Expired Permit': 'CANCELLED',
        'Expired, Not Released': 'CANCELLED',
        'Not Completed': 'PERMITTED',
        'Other': 'UNKNOWN',
        'P&A - Approved': 'PA',
        'Permit to Drill': 'PERMITTED',
        'Permitted Injection Well': 'PERMITTED',
        'Producing': 'ACTIVE',
        'Revoked Inj. Permit': 'CANCELLED',
        'Shut In': 'SI',
        'Spudded': 'ACTIVE',
        'Temporarily Abandoned': 'TA',
        'Unknown': 'UNKNOWN',
        'Water Well, Completed': 'ACTIVE',
        'Water Well, Released': 'ACTIVE',
    },
    'type_map': {
        'Class V Injection': 'INJECTION',
        'Coal Bed Methane': 'GAS',
        'Domestic Water': 'WATER',
        'Dry Hole': 'DRY HOLE',
        'Expired Location': 'UNKNOWN',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Injection - Disposal': 'DISPOSAL',
        'Injection, EOR': 'INJECTION',
        'Injection, Indian Lands': 'INJECTION',
        'Monitor/Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Oil & Gas': 'OILANDGAS',
        'Stratigraphic Test': 'TEST',
        'Unknown': 'UNKNOWN',
        'Water Source': 'WATER',
    },
}
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

44513 of 44534 rows loaded
first item: [{'state': 'MT', 'api': '25009052950000', 'lon': -108.7862105, 'lat': 45.53405547, 'date': datetime.date(2051, 6, 1), 'type': 'DRY HOLE', 'status': 'PA', 'category': None, 'description': None}]
loaded 44513 of 44513 records into the database


44513

In [23]:
# Nebraska
state = fieldmaps.State('NE')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Abandoned': 'A',
        'Abandoned:Dry Hole': 'A',
        'Abandoned:Plugged': 'PA',
        'Active': 'ACTIVE',
        'Active:Injecting': 'ACTIVE',
        'Active:Producing': 'ACTIVE',
        'Inactive': 'A',
        'Inactive:Shut In': 'SI',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Active': 'ACTIVE',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Unassigned': 'UNKNOWN',
        'Unknown': 'UNKNOWN',
    },
    'type_map': {
        'Abandoned': 'UNKNOWN',
        'Dry Hole': 'DRY HOLE',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Observation Monitor Strat Test': 'TEST',
        'Oil': 'OIL',
        'Other': 'OTHER',
        'Permit': 'UNKNOWN',
        'Unassigned': 'UNKNOWN',
        'Underground Injection Control:Class 1': 'INJECTION',
        'Underground Injection Control:Class 2': 'INJECTION',
        'Underground Injection Control:Class 2 Disposal': 'DISPOSAL',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Water Supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

20194 of 22253 rows loaded
first item: [{'state': 'NE', 'api': '26105074140001', 'lon': -104.053, 'lat': 41.2764, 'date': datetime.date(1981, 5, 2), 'type': 'UNKNOWN', 'status': 'A', 'category': None, 'description': None}]
loaded 20194 of 20194 records into the database


20194

In [26]:
# Nevada
state = fieldmaps.State('NV') # strip_whitespace = ['Well Status', 'Well Type']
state.projection = 26910
layout = {
    'api_field': 'API',
    'date_field': 'COMPL_DATE',
    'lat_field': 'APROX_UTMN',
    'lon_field': 'APROX_UTME',
    'status_field': 'STATUS',
    'status_map': {
        ' ': 'UNKNOWN',
        'A': 'A',
        'Abandoned': 'A',
        'Cancelled': 'CANCELLED',
        'D & A': 'A',
        'D & A (?)': 'A',
        'D & A ?': 'A',
        'D&A': 'A',
        'D&A(?)': 'A',
        'D&A,Water well': 'A',
        'Drilled': 'ACTIVE',
        'Drilled?': 'ACTIVE',
        'Expired': 'CANCELLED',
        'Geothermal': 'ACTIVE',
        'Geothermal Wel': 'ACTIVE',
        'Injection': 'ACTIVE',
        'J & A': 'A',
        'J & A ?': 'A',
        'J&A ?': 'A',
        'Lease closed': 'CANCELLED',
        'Never Drilled': 'CANCELLED',
        'Never drilled': 'CANCELLED',
        'Never drilled?': 'CANCELLED',
        'P & A': 'PA',
        'P & A (?)': 'PA',
        'P & A 7/17/95': 'PA',
        'P & A 7/27/95': 'PA',
        'P&A': 'PA',
        'P&A (Producer)': 'PA',
        'Permit Revoked': 'CANCELLED',
        'Prod?Inject': 'ACTIVE',
        'Producer': 'ACTIVE',
        'Producer(?)': 'ACTIVE',
        'Shut In': 'SI',
        'Shut in': 'SI',
        'Suspended': 'TA',
        'TA': 'TA',
        'TA ?': 'TA',
        'Temp. Shut In': 'TA',
        'Testing': 'ACTIVE',
        'Water Disposal': 'ACTIVE',
        'Water Well': 'ACTIVE',
        'Water well': 'ACTIVE',
        'drilled': 'ACTIVE',
        'injection': 'ACTIVE',
        'injection well': 'ACTIVE',
        'producer': 'ACTIVE',
        're-entry': 'ACTIVE',
        'water well': 'ACTIVE',
    },
    'type_field': 'SHOW',
    'type_map': {
        ' ': 'UNKNOWN',
        '?': 'UNKNOWN',
        'GAS': 'GAS',
        'GAS  OIL': 'OILANDGAS',
        'GAS  WATER': 'GAS',
        'GAS WATER': 'GAS',
        'OIL': 'OIL',
        'OIL   GAS': 'OILANDGAS',
        'OIL  GAS': 'OILANDGAS',
        'OIL  GAS  WATER': 'OILANDGAS',
        'OIL  WATER': 'OIL',
        'OIL  WATER  GAS': 'OILANDGAS',
        'OIL  `': 'OIL',
        'OIL GAS': 'OIL',
        'OIL GAS WATER': 'OILANDGAS',
        'OIL WATER': 'OIL',
        'OIL, WATER': 'OIL',
        'OIL?': 'OIL',
        'Oil': 'OIL',
        'WATER': 'WATER',
        'WATER  GAS': 'GAS',
        'WATER  HELIUM G': 'OTHER',
        'WATER  OIL': 'OIL',
        'WATER  OIL GAS': 'OILANDGAS',
        'Water': 'WATER',
        'weak oil': 'OIL',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

String does not contain a date.
Error converting date for row:

OrderedDict([('', '1'), ('AL_BOT', '99999.0'), ('API', '  27-003-05057'), ('APROX_UTME', '653587.0'), ('APROX_UTMN', '3987300.0'), ('COMPL_DATE', ' '), ('COUNTY', 'Clark'), ('DISTFROMLI', '200 S,200 W'), ('ELEVATION', '  2,800'), ('LOGS', 'Drillers Log'), ('OBJECTID', '2'), ('OBJECTID_1', ''), ('OIL_FIELD', ' '), ('OPERATOR', 'Hatfield, L.M.'), ('PARTSECT', 'SW/4 SW/4 NW/4'), ('PERMIT', '0.0'), ('PERMIT_ISS', '31 JUL 35'), ('PLOT_SYM', 'Dry Hole'), ('RANGE', '  60E'), ('REMARKS', 'Drillers Log from Lintz, 1957, p. 23-24.'), ('SAMPLES', ' '), ('SECTION_', '20'), ('SHOW', ' '), ('STATUS', 'A'), ('TD', '   707'), ('TD_NUMERIC', '707.0'), ('TESTS', ' '), ('TOPS', ' '), ('TOWNSHIP', '  22S'), ('WELL', 'No. 1')])
781 of 1842 rows loaded
first item: [{'state': 'NV', 'api': '2700105054', 'lon': 367985.0, 'lat': 4369112.0, 'date': datetime.date(2024, 9, 12), 'type': 'OILANDGAS', 'status': 'PA', 'category': None, 'description': None

781

In [ ]:
# New Hampshire - no oil or gas

In [ ]:
# New Jersey - no oil or gas

In [32]:
# New Mexico
# I think the data is actually in WGS84 b/c that's how I requested
# it from the NM ArcGIS server
# status map: ftp://164.64.106.6/Public/OCD/OCD%20GIS%20Data/Metadata/NM_Wells_Metadata.pdf
state = fieldmaps.State('NM')
layout = {
    'api_field': 'API',
    'date_field': 'spud_date',
    'lat_field': 'latitude',
    'lon_field': 'longitude',
    'status_field': 'status',
    'status_map': {
        '0': 'UNKNOWN',
        'A': 'ACTIVE',
        'C': 'CANCELLED',
        'D': 'A',
        'E': 'CANCELLED',
        'H': 'A',
        'N': 'PERMITTED',
        'P': 'PA',
        'Q': 'PA',
        'S': 'SI',
        'T': 'TA',
        'X': 'PERMITTED',
        'Z': 'TA',
    },
    'type_field': 'well_type',
    'type_map': {
        'C': 'OILANDGAS',
        'G': 'GAS',
        'I': 'INJECTION',
        'M': 'OTHER',
        'O': 'OIL',
        'S': 'DISPOSAL',
        'W': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

117927 of 117927 rows loaded
first item: [{'state': 'NM', 'api': '3001501065', 'lon': -104.2108307, 'lat': 32.5064774, 'date': datetime.date(9999, 12, 31), 'type': 'OIL', 'status': 'PA', 'category': None, 'description': None}]
loaded 117927 of 117927 records into the database


117927

In [34]:
# New York
state = fieldmaps.State('NY', webdate = True)
state.projection = 4269 # NAD83
# STATUS CODES LOOKUP https://www.dec.ny.gov/energy/30438.html
layout = {
    'api_field': 'API_WELLNO',
    'date_field': ['DATE_SPUDDED', 'DATE_PERMIT_APPLICATION', 'DATE_WELL_COMPLETED'],
    'lat_field': 'SURFACE_LATITUDE',
    'lon_field': 'SURFACE_LONGITUDE',
    'status_field': 'WELL_STATUS',
    'status_map': {
        'AC': 'ACTIVE',
        'AR': 'UNKNOWN',
        'CA': 'CANCELLED',
        'CO': 'ACTIVE',
        'Confidential': 'ACTIVE',
        'DC': 'ACTIVE',
        'DD': 'ACTIVE',
        'DG': 'ACTIVE',
        'EX': 'CANCELLED',
        'IN': 'INACTIVE',
        'NR': 'UNKNOWN',
        'PA': 'PA',
        'PB': 'PA',
        'PI': 'PERMITTED',
        'PM': 'PA',
        'RE': 'CANCELLED',
        'SI': 'SI',
        'TA': 'TA',
        'TR': 'UNKNOWN',
        'UL': 'UNKNOWN',
        'UM': 'UNKNOWN',
        'UN': 'UNKNOWN',
        'VP': 'CANCELLED',
    },
    'type_field': 'WELL_TYPE',
    'type_map': {
        'BR': 'WATER',
        'Confidential': 'UNKNOWN',
        'DH': 'DRY HOLE',
        'DS': 'DISPOSAL',
        'DW': 'DRY HOLE',
        'GD': 'GAS',
        'GE': 'GAS',
        'GW': 'GAS',
        'IW': 'INJECTION',
        'LP': 'STORAGE',
        'MB': 'OBSERVATION',
        'MM': 'OBSERVATION',
        'MS': 'OBSERVATION',
        'NL': 'OTHER',
        'OD': 'OIL',
        'OE': 'OIL',
        'OW': 'OIL',
        'SG': 'TEST',
        'ST': 'STORAGE',
        'TH': 'OTHER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

41802 of 41802 rows loaded
first item: [{'state': 'NY', 'api': '31013116250000', 'lon': -79.42551999999999, 'lat': 42.37863, 'date': datetime.date(1975, 10, 1), 'type': 'GAS', 'status': 'ACTIVE', 'category': None, 'description': None}]
loaded 41802 of 41802 records into the database


41802

In [ ]:
# North Carolina - no oil or gas

In [36]:
# North Dakota
state = fieldmaps.State('ND')
# code definitions: https://www.dmr.nd.gov/oilgas/codehelp.asp
layout = {
    'api_field': 'api',
    'date_field': 'spud_date',
    'lat_field': 'latitude',
    'lon_field': 'longitude',
    'status_field': 'status',
    'status_map': {
        'A': 'ACTIVE',
        'AB': 'A',
        'Confidential': 'ACTIVE',
        'DRL': 'ACTIVE',
        'DRY': 'DRY',
        'EXP': 'CANCELLED',
        'IA': 'INACTIVE',
        'IAW': 'INACTIVE',
        'LOC': 'PERMITTED',
        'LOCR': 'PERMITTED',
        'NC': 'PERMITTED',
        'NCW': 'PERMITTED',
        'PA': 'PA',
        'PNC': 'CANCELLED',
        'PNS': 'CANCELLED',
        'TA': 'TA',
        'TAO': 'TA',
        'TASC': 'TA',
        'TATD': 'TA',
    },
    'type_field': 'well_type',
    'type_map': {
        'AGD': 'DISPOSAL',
        'AI': 'INJECTION',
        'CBM': 'GAS',
        'Confidential': 'UNKNOWN',
        'DF': 'INJECTION',
        'DFP': 'INJECTION',
        'GASC': 'GAS',
        'GASD': 'GAS',
        'GASN': 'GAS',
        'GI': 'INJECTION',
        'GS': 'STORAGE',
        'INJP': 'INJECTION',
        'OG': 'OILANDGAS',
        'ST': 'TEST',
        'SWD': 'DISPOSAL',
        'WI': 'INJECTION',
        'WS': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Error converting status for row:

OrderedDict([('file_no', '34479'), ('api_no', '33-105-04682-00-00'), ('operator', 'WHITING OIL AND GAS CORPORATION'), ('well_name', 'MOEN  41-2-5H'), ('td', '0'), ('spud_date', ''), ('field_name', 'STOCKYARD CREEK'), ('qq', 'LOT1'), ('sec', '2'), ('twp', '154'), ('rng', '99'), ('feet_ns', '365'), ('fnsl', 'N'), ('feet_ew', '570'), ('fewl', 'E'), ('latitude', '48.19687506'), ('longitude', '-103.36874557'), ('well_type', 'OG'), ('status', ''), ('symbol', ''), ('api', '33105046820000'), ('datecreate', ''), ('datemodifi', ''), ('scribe', ''), ('County', 'WILLIAMS')])
35153 of 35154 rows loaded
first item: [{'state': 'ND', 'api': '33029000020000', 'lon': -100.190907, 'lat': 46.289914, 'date': datetime.date(1950, 2, 19), 'type': 'OILANDGAS', 'status': 'DRY', 'category': None, 'description': None}]
loaded 35153 of 35153 records into the database


35153

In [6]:
# Ohio
state = fieldmaps.State('OH', date_function = lambda x: None)

layout = {
    'api_field': 'api',
    'date_field': ['commenced', 'completed', 'issued'],
    'lat_field': 'WH_LAT',
    'lon_field': 'WH_LONG',
    'status_field': 'WL_STATUS_DESC',
    'status_map': {
        'Active Injection': 'ACTIVE',
        'Cancelled': 'CANCELLED',
        'Domestic Well': 'ACTIVE',
        'Drilled, Inactive': 'INACTIVE',
        'Drilling': 'ACTIVE',
        'Dry and Abandoned': 'A',
        'Exempt Mississippian Well': 'UNKNOWN',
        'Field Inspected, Well Not Found': 'UNKNOWN',
        'Final Restoration': 'PA',
        'Historical Production Well': 'UNKNOWN',
        'Lost Hole': 'UNKNOWN',
        'Not Drilled': 'CANCELLED',
        'Orphan Well - Pending': 'ORPHAN',
        'Orphan Well - Ready': 'ORPHAN',
        'Plugged Back': 'PA',
        'Plugged and Abandoned': 'PA',
        'Producing': 'ACTIVE',
        'Replugged Well': 'PA',
        'Shut in': 'SI',
        'Storage Well': 'ACTIVE',
        'Temporary Inactive Well Status 1st Year': 'INACTIVE',
        'Temporary Inactive Well Status 2nd Year': 'INACTIVE',
        'Temporary Inactive Well Status 3rd Year': 'INACTIVE',
        'Unknown status': 'UNKNOWN',
        'Well Drilled': 'ACTIVE',
        'Well Permitted': 'PERMITTED',
    },
    'type_field': 'MapSymbol_DESC',
    'type_map': {
        'Brine for dust control': 'WATER',
        'Coalbed methane': 'GAS',
        'Dry hole': 'DRY HOLE',
        'Dry hole with gas show': 'GAS',
        'Dry hole with oil and gas show': 'GAS',
        'Dry hole with oil show': 'OIL',
        'Expired permit': 'UNKNOWN',
        'Gas': 'GAS',
        'Gas and oil show': 'OILANDGAS',
        'Gas show': 'GAS',
        'Gas storage': 'GAS',
        'Gas with oil show': 'OILANDGAS',
        'Injection': 'INJECTION',
        'Lost hole': 'UNKNOWN',
        'Observation': 'OBSERVATION',
        'Oil': 'OIL',
        'Oil and gas': 'OILANDGAS',
        'Oil and gas converted to water': 'WATER',
        'Oil show': 'OIL',
        'Oil with gas show': 'OILANDGAS',
        'Permitted location': 'UNKNOWN',
        'Plugged brine for dust control': 'WATER',
        'Plugged gas': 'GAS',
        'Plugged gas with oil show': 'OILANDGAS',
        'Plugged injection': 'INJECTION',
        'Plugged oil': 'OIL',
        'Plugged oil and gas': 'OILANDGAS',
        'Plugged oil with gas show': 'OILANDGAS',
        'Plugged water supply': 'WATER',
        'Plugged with gas show': 'GAS',
        'Plugged with oil show': 'OIL',
        'Radioactive tools lost in hole': 'UNKNOWN',
        'Solution mining': 'UNKNOWN',
        'Stratigraphy test': 'TEST',
        'Unknown status': 'UNKNOWN',
        'Water supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Error converting status for row:

OrderedDict([('', '0'), ('ADDR1', '2045 Morse Rd., Bldg F-2'), ('ADDR2', ''), ('API_WELLNO', '34001200010000'), ('API_WELLNO_LINK', '34001200010000'), ('BH_LAT', ''), ('BH_LONG', ''), ('CITY', 'columbus'), ('CO_NAME', 'UNREGISTERED OWNER'), ('MapSymbol', 'dry_gasshow      '), ('MapSymbol_DESC', 'Dry hole with gas show'), ('Marcellus_Shale', '0'), ('OBJECTID', '1'), ('PHONE', ''), ('ProducingFormation1', ''), ('ProducingFormation2', ''), ('ProposedFormation', ''), ('SLANT', 'V'), ('Utica_Shale', '0'), ('WELL_NM', 'ELLISON FRANK 1'), ('WELL_NO', '1'), ('WH_LAT', '38.70370901154379'), ('WH_LONG', '-83.60375219491999'), ('WL_CNTY', 'ADAMS'), ('WL_STATUS', ''), ('WL_STATUS_DESC', ''), ('WL_TWP', 'SPRIGG'), ('ZIP1', '43229'), ('api', '34001200010000'), ('commenced', '10/6/1958'), ('completed', '11/15/1958'), ('issued', '8/8/1958')])
230298 of 235674 rows loaded
first item: [{'state': 'OH', 'api': '34001200020000', 'lon': -83.52674019799007, 'lat': 38.7132518

230298

In [48]:
# Oklahoma
# about half of Oklahoma's older wells are in TRS coordinates. I know
# once upon a time I had a function to convert from TRS to decimal degrees
# for the time being, these are getting ignored.
# also, unclear if this dataset includes the reservation or not.
state = fieldmaps.State('OK')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': {
        'Abandoned:Plugged': 'PA',
        'Active': 'ACTIVE',
        'Active:Completed': 'ACTIVE',
        'Inactive:Temporarily Abandoned': 'TA',
        'Permit:Active': 'ACTIVE',
        'Permit:Cancelled Expired': 'CANCELLED',
        'Unassigned': 'UNKNOWN',
        'Unknown': 'UNKNOWN',
    },
    'type_map': {
        'Abandoned': 'UNKNOWN',
        'Dry Hole': 'DRY HOLE',
        'Gas': 'GAS',
        'Gas Storage': 'STORAGE',
        'Oil': 'OIL',
        'Oil and Gas': 'OILANDGAS',
        'Other': 'OTHER',
        'Unassigned': 'UNKNOWN',
        'Underground Injection Control:Class 2': 'INJECTION',
        'Underground Injection Control:Class 2 Disposal': 'DISPOSAL',
        'Underground Injection Control:Class 2 Enhanced Recovery': 'INJECTION',
        'Unknown': 'UNKNOWN',
        'Water Supply': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

250247 of 533003 rows loaded
first item: [{'state': 'OK', 'api': '35139354890000', 'lon': -101.963, 'lat': 36.8249, 'date': datetime.date(1954, 11, 8), 'type': 'OIL', 'status': 'PA', 'category': None, 'description': None}]
loaded 250247 of 250247 records into the database


250247

In [53]:
# Oregon
state = fieldmaps.State('OR')

layout = {
    'api_field': 'PermitID',
    'date_field': 'ApplicationDate',
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'status_field': 'Status',
    'status_map': {
        'Application': 'UNKNOWN',
        'Application Withdrawn': 'CANCELLED',
        'Cancelled': 'CANCELLED',
        'Closed': 'A',
        'Denied': 'CANCELLED',
        'Extended to 11/24/83': 'UNKNOWN',
        'Extended to 5/28/93': 'UNKNOWN',
        'Extended to 7/25/87': 'UNKNOWN',
        'Permitted': 'PERMITTED',
        'Terminated': 'A',
        'Unknown': 'UNKNOWN',
        'Withdrawn': 'CANCELLED',
    },
    'type_field': 'WellType',
    'type_map': {
        'Dry Hole': 'DRY HOLE',
        'Exploration': 'TEST',
        'Former producer - plugged': 'OIL',
        'Gas Injection/Withdrawal': 'INJECTION',
        'Gas Injection/Withdrawl': 'INJECTION',
        'Information': 'OBSERVATION',
        'Monitoring': 'OBSERVATION',
        'Never produced - plugged ': 'OIL',
        'Not Drilled': 'UNKNOWN',
        'Not drilled': 'UNKNOWN',
        'Observation': 'OBSERVATION',
        'Observation well': 'OBSERVATION',
        'Producer': 'OIL',
        'Water Disposal': 'WATER',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

714 of 723 rows loaded
first item: [{'state': 'OR', 'api': '3600100003', 'lon': -117.7791061, 'lat': 44.8982391, 'date': datetime.date(2052, 9, 24), 'type': 'OTHER', 'status': 'UNKNOWN', 'category': None, 'description': None}]
loaded 714 of 714 records into the database


714

In [54]:
# Pennsylvania
state = fieldmaps.State('PA')
state.source_url = 'http://www.depreportingservices.state.pa.us/ReportServer/Pages/ReportViewer.aspx?/Oil_Gas/Spud_External_Data'
state.description = """
Data downloaded in CSV form based on setting SPUD BEGIN DATE = 1/1/1800 and SPUD END DATE = 12/31/2099
Represents 142,788, of which 110,000 have dates wells.
"""
state.write_to_db()

layout = {
    'date_field': 'SPUD_DATE',
    'api_field': 'API',
    'type_field': 'WELL_CODE_DESC',
    'status_field': 'WELL_STATUS',
    'category_field': 'UNCONVENTIONAL',
    'lat_field': 'LATITUDE',
    'lon_field': 'LONGITUDE',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'OGO No: %s',
                             'Municipality: %s',
                             'Farm Name: %s',
                             'Well Configuration: %s'
                            ]),
    'description_fields': ['COUNTY', 'OPERATOR', 'OGO_NUM', 'MUNICIPALITY', 'FARM_NAME', 'CONFIGURATION'],
    'type_map': {
        'UNDETERMINED': 'OTHER',
        'MULTIPLE WELL BORE TYPE': 'OTHER',
        'OIL': 'OIL',
        'OBSERVATION': 'OBSERVATION',
        'DRY HOLE': 'DRY HOLE',
        'COALBED METHANE': 'GAS',
        'GAS': 'GAS',
        'COMB. OIL&GAS': 'OILANDGAS',
        'STORAGE WELL': 'STORAGE',
        'WASTE DISPOSAL': 'DISPOSAL',
        'INJECTION': 'INJECTION',
        'TEST WELL': 'TEST',
    },

    'status_map': {
        'DEP Abandoned List': 'ORPHAN',
        'Plugged OG Well': 'PA',
        'DEP Orphan List': 'ORPHAN',
        'DEP Plugged': 'PA',
        'Regulatory Inactive Status': 'UNKNOWN',
        'Operator Reported Not Drilled': 'CANCELLED',
        'Proposed But Never Materialized': 'CANCELLED',
        'Active': 'ACTIVE',
        'Abandoned': 'PA',
    },

    'category_map': {
        'Yes': 'UNCONVENTIONAL',
        'No': 'CONVENTIONAL',
    },

}

wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Wrote description for PA to database
143936 of 204142 rows loaded
first item: [{'state': 'PA', 'api': '00300921', 'lon': -80.139878, 'lat': 40.529608, 'date': None, 'type': 'OIL', 'status': 'ACTIVE', 'category': 'CONVENTIONAL', 'description': 'County: Allegheny\nOperator Name: DANIEL J NADLER\nOGO No: OGO-32546\nMunicipality: Aleppo Twp\nFarm Name: ZACHARY NADLER 1\nWell Configuration: Vertical Well'}]
loaded 143936 of 143936 records into the database


143936

In [ ]:
# Rhode Island - no oil or gas

In [ ]:
# South Carolina - No oil or gas

In [60]:
# South Dakota
state = fieldmaps.State('SD')
state.projection = 4269 # NAD83
layout = {
    'api_field': 'API Number',
    'date_field': ['Spud Date', 'Date Permit Issued'],
    'lat_field': 'Latitude (GCS83)',
    'lon_field': 'Longitude (GCS83)',
    'status_field': 'Administrative Status',
    'status_map': {
        'Abandoned-Not Regulated': 'A',
        'Converted-Domestic Gas': 'ACTIVE',
        'Converted-Fresh Water Supply': 'ACTIVE',
        'Converted-Water Supply-Injection': 'ACTIVE',
        'Injecting': 'ACTIVE',
        'Never drilled': 'CANCELLED',
        'Not Spudded': 'PERMITTED',
        'Plugged and Abandoned': 'PA',
        'Producing': 'ACTIVE',
        'Shut In': 'SI',
        'Temporarily Abandoned': 'TA',
    },
    'type_field': 'Permitted Well Type',
    'type_map': {
        'Gas': 'GAS',
        'Injection': 'INJECTION',
        'Injection-Horizontal': 'INJECTION',
        'Oil': 'OIL',
        'Oil and Gas Exploration': 'TEST',
        'Oil-Horizontal': 'OIL',
    },
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

2104 of 2104 rows loaded
first item: [{'state': 'SD', 'api': '4000305001', 'lon': -98.403639, 'lat': 43.788912, 'date': datetime.date(2045, 5, 28), 'type': 'TEST', 'status': 'PA', 'category': None, 'description': None}]
loaded 2104 of 2104 records into the database


2104

In [61]:
# Tennessee
state = fieldmaps.State('TN')

state.description = """
Tennessee's Department of Environment & Conservation does not 
doesn't make a lot of details available (no spud date, well status). The dates recorded are the 
Permit Date. Status is set to unknown for all wells. Per their website, 
<i>"This minimal permit information is intended to allow the public to easily track the number and 
locations of oil and gas well permits issued. For more detailed information about a specific well, 
contact the Oil and Gas program staff at 615-687-7109 . Permit files can be accessed by visiting 
the Oil and Gas Section at the Nashville Environmental Field Office in Nashville, TN 37243."</i>
"""
state.source_url = 'http://environment-online.state.tn.us:8080/pls/enf_reports/f?p=9034:34300:0::NO:::'
state.write_to_db()

layout = {
    'date_field': 'Permit Date',
    'api_field': 'API No',
    'type_field': 'Purpose af Well',
    'status_field': 'Purpose af Well',
    'lat_field': 'Latitude',
    'lon_field': 'Longitude',
    'description_layout': '\n'.join(['County: %s',
                             'Operator Name: %s',
                             'Permit No: %s',
                             'Well Name and Number: %s',
                             'EFO Name: %s'
                            ]),
    'description_fields': ['County', 'Operator Name', 'Permit No', 'Well Name and Number', 'EFO Name'],
    'type_map': {
        'Gas': 'GAS',
        'Mineral Core Test': 'TEST',
        'NCG - Domestic Use': 'GAS',
        'Oil': 'OIL',
        'Oil And Gas': 'OILANDGAS',
    },
    'status_map': {
        'Gas': 'ACTIVE',
        'Mineral Core Test': 'UNKNOWN',
        'NCG - Domestic Use': 'ACTIVE',
        'Oil': 'ACTIVE',
        'Oil And Gas': 'ACTIVE',
    },
}

wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Wrote description for TN to database
Error converting status for row:

OrderedDict([('County', 'Overton'), ('EFO Name', 'Cookeville'), ('Permit Date', '21-MAR-1994'), ('Permit No', '0008788'), ('API No', '133-21202'), ('Operator Name', 'Southeastern Oil Corp./Smithco'), ('Well Name and Number', 'BROWN WAYNE #2'), ('Latitude', '36.430083'), ('Longitude', '-85.324472'), ('Elevation', '1368'), ('Formation at Total Depth', ''), ('Purpose af Well', '')])
10185 of 16144 rows loaded
first item: [{'state': 'TN', 'api': '02720603R1', 'lon': -85.340056, 'lat': 36.533389, 'date': datetime.date(2018, 2, 23), 'type': 'OIL', 'status': 'ACTIVE', 'category': None, 'description': 'County: Clay\nOperator Name: John Henry Oil Corporation\nPermit No: 0012939\nWell Name and Number: W. C. Vaughn #2\nEFO Name: Cookeville'}]
loaded 10185 of 10185 records into the database


10185

In [64]:
# Texas
# ug - no date field
state = fieldmaps.State('TX', prepend_api = True, date_function = lambda x: None)
state.projection = 4269 # NAD83
layout = {
    'api_field': 'API',
    'date_field': 'API',
    'lat_field': 'GIS_LAT83',
    'lon_field': 'GIS_LONG83',
    'status_field': 'SYMNUM',
    'status_map': None,
    'type_field': 'GIS_SYMBOL_DESCRIPTION',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, layout)
wells.write_to_db()

Missing or invalid type_map.
valid options: {'WATER', 'OBSERVATION', 'GAS', 'TEST', 'OIL', 'UNKNOWN', 'SERVICE', 'OTHER', 'STORAGE', 'OILANDGAS', 'DRY HOLE', 'INJECTION', 'DISPOSAL'}
    'type_map': {
        'Brine Mining': '',
        'Brine Mining / Oil': '',
        'Brine Mining / Oil / Gas': '',
        'Canceled / Abandoned Location': '',
        'Core Test': '',
        'Dry Hole': 'DRY HOLE',
        'Gas Well': '',
        'Geothermal Well': '',
        'Injection / Disposal': '',
        'Injection / Disposal from Gas': '',
        'Injection / Disposal from Oil': '',
        'Injection / Disposal from Oil/Gas': '',
        'Injection/Disposal from Brine Mining': '',
        'Injection/Disposal from Brine Mining / Gas': '',
        'Injection/Disposal from Storage': '',
        'Injection/Disposal from Storage/Brine Mining': '',
        'Injection/Disposal from Storage/Gas': '',
        'Injection/Disposal from Storage/Oil': '',
        'Observation Well': '',
        'Obser

False

In [5]:
importlib.reload(fieldmaps)

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}

In [13]:
# Utah
state = fieldmaps.State('UT')

layout = {
   'date_field': ['SPUD DATE', 'FIRST COMPLETION DATE', 'FIRST PRODUCTION DATE'],
    'api_field': 'API NUMBER',
    'type_field': 'STANDARD WELL TYPE DESCRIPTION',
    'status_field': 'STANDARD WELL STATUS DESCRIPTION',
    'lat_field': 'WELLHEAD LATITUDE, DECIMAL DEGREES',
    'lon_field': 'WELLHEAD LONGITUDE, DECIMAL DEGREES',
    'status_map': None,
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

Missing layout.

{'api_field': '', 'date_field': '', 'lat_field': '', 'lon_field': '', 'status_field': '', 'status_map': None, 'type_field': '', 'type_map': None}
Invalid layout. Available source file columns: 
'X', 'Y', 'API', 'WellName', 'Operator', 'OperatorNo', 'FieldName', 'Ground_Ele', 'Kelly_Elev', 'DrkFloor_E', 'CoordsSurf', 'CoordsSu_1', 'UTMZone', 'Latitude', 'Longitude', 'FootageNS', 'Dir_NS', 'FootageEW', 'Dir_EW', 'QtrQtr', 'Section', 'Township', 'TownshipDi', 'Range', 'RangeDir', 'Meridian', 'County', 'Dir_Horiz', 'Dir_Vert', 'Dir_Direct', 'Confidenti', 'ConfRelDat', 'LeaseNumbe', 'LeaseType', 'SurfaceOwn', 'AbandonDat', 'WellStatus', 'WellType', 'TotCum_Oil', 'TotCum_Gas', 'TotCum_Wat', 'IndianTrib', 'Multi_Lats', 'OriginalFi', 'UnitName', 'GISStatusT', 'OrigComplD', 'JURISDICTI'

Example rows: [OrderedDict([('X', '-109.838316612021'), ('Y', '40.3801299206766'), ('API', '4304710581'), ('WellName', 'ROSEMARY LLOYD 1'), ('Operator', 'HUMBLE OIL & REFINING CO'), ('OperatorN

False

In [ ]:
# Vermont - no oil or gas

In [12]:
# Virginia
state = fieldmaps.State('VA')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

NameError: name 'filename' is not defined

In [11]:
# Washington
state = fieldmaps.State('WA')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

NameError: name 'filename' is not defined

In [10]:
# West Virginia
state = fieldmaps.State('WV')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

NameError: name 'filename' is not defined

In [ ]:
# Wisconsin - no oil or gas

In [9]:
# Wyoming
state = fieldmaps.State('WY')

layout = {
    'api_field': '',
    'date_field': '',
    'lat_field': '',
    'lon_field': '',
    'status_field': '',
    'status_map': None,
    'type_field': '',
    'type_map': None
}
#import fieldmaps.ks
wells = fieldmaps.Dataset(state, )
wells.write_to_db()

NameError: name 'filename' is not defined

In [ ]:
# Alaska Offshore
state = State('AK1')
# Pacific Coast Offshore
state = State('CA1')
# Northern Gulf of Mexico
state = State('TX1')
# Atlantic Coast Offshore
state = State('DC1')
